In [1]:
import pennylane as qml
from pennylane import numpy as np

## P.2.1

Concept: using multiple control wires simultaneously to do simultaneous' measurement.

Concept: suppose the unitary on the eigenvector in last section's phase estimation circuit resulted in a pure state in the control wire; the control wire being $\ket{0}$ or $\ket{1}$ at the end told us which eigenvalue that eigenvector corresponded to.

Concept: to generalize to multiple control wires, the Hadamard on one control wire is generalized to the adjoint QFT on multiple control wires. In this setup, we are able to talk about the control wire being set to $\ket{x}$ where $x$ is a bit pattern in the number of control wires. Furthermore, we apply some power of the unitary instead on each estimation wire to extract the corresponding position in the bit pattern.

In [ ]:
def U_power_2k(unitary, k):
    """ Computes U at a power of 2k (U^2^k)
    
    Args: 
        unitary (array[complex]): A unitary matrix
    
    Returns: 
        array[complex]: the unitary raised to the power of 2^k
    """
    ##################
    # YOUR CODE HERE #
    ##################  
    return np.linalg.matrix_power(unitary, 2**k)

# Try out a higher power of U
U = qml.T.compute_matrix()
print(U)

U_power_2k(U, 2)


## P.2.2


In [ ]:
estimation_wires = [0, 1, 2]
target_wires = [3]

def apply_controlled_powers_of_U(unitary):
    """A quantum function that applies the sequence of powers of U^2^k to
    the estimation wires.
    
    Args: 
        unitary (array [complex]): A unitary matrix
    """

    ##################
    # YOUR CODE HERE #
    ##################
    for k in estimation_wires:
        qml.ControlledQubitUnitary(U_power_2k(unitary, len(estimation_wires) - k - 1), control_wires=k, wires=target_wires)

## P.2.3


In [ ]:
dev = qml.device("default.qubit", wires=4)

estimation_wires = [0, 1, 2]
target_wires = [3]

def prepare_eigenvector():
    qml.PauliX(wires=target_wires)

@qml.qnode(dev)
def qpe(unitary):
    """ Estimate the phase for a given unitary.
    
    Args:
        unitary (array[complex]): A unitary matrix.
        
    Returns:
        array[float]: Measurement outcome probabilities on the estimation wires.
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)
    prepare_eigenvector()
    apply_controlled_powers_of_U(unitary)
    qml.adjoint(qml.QFT)(wires=estimation_wires)
    return qml.probs(wires=estimation_wires)

U = qml.T.compute_matrix()
print(qpe(U))


## P.2.4

In [ ]:
estimation_wires = [0, 1, 2]
target_wires = [3]

def estimate_phase(probs):
    """Estimate the value of a phase given measurement outcome probabilities
    of the QPE routine.
    
    Args: 
        probs (array[float]): Probabilities on the estimation wires.
    
    Returns:
        float: the estimated phase   
    """
    ##################
    # YOUR CODE HERE #
    ##################
    return np.argmax(probs) / 2 ** len(estimation_wires)

U = qml.T.compute_matrix()

probs = qpe(U)


estimated_phase = estimate_phase(probs)
print(estimated_phase)


## P.2.5

In [ ]:
dev = qml.device("default.qubit", wires=4)

estimation_wires = [0, 1, 2]
target_wires = [3]

def prepare_eigenvector():
    qml.PauliX(wires=target_wires)

@qml.qnode(dev)
def qpe(unitary):
    """Estimate the phase for a given unitary.
    
    Args:
        unitary (array[complex]): A unitary matrix.
        
    Returns:
        array[float]: Probabilities on the estimation wires.
    """
    
    prepare_eigenvector()
    
    ##################
    # YOUR CODE HERE #
    ##################
    qml.QuantumPhaseEstimation(unitary, target_wires=target_wires, estimation_wires=estimation_wires)
    return qml.probs(wires=estimation_wires)


U = qml.T.compute_matrix()
probs = qpe(U)
print(estimate_phase(probs))
